In [4]:
import numpy as np
import tensorflow as tf

# Paso 1: Obtener y preprocesar el texto

path = "pg4300.txt"
with open(path) as f:
    text = f.read()

# Preprocesar texto (simplificado, considerar limpiar encabezados y pies de página del sitio)
text = text.lower()  # Convertir a minúsculas
# Crear secuencias de caracteres y sus próximos caracteres
maxlen = 50
step = 5

# Crear una lista de los caracteres únicos en el texto
chars = sorted(list(set(text)))

# Crear un diccionario que mapee cada carácter a un número entero único
char_to_int = dict((c, i) for i, c in enumerate(chars))

def sample(preds, temperature=1.0):
    """Función de muestreo que incorpora la diversidad."""
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length, diversity=1.0):
    start_index = np.random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence

    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_int[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = chars[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

    return generated

model = tf.keras.models.load_model('ulisses_0.2_token.keras')

print(generate_text(150, 1.0))
print('---------------------------')
print(generate_text(50, 0.5))
print('---------------------------')
print(generate_text(50, 0.2))

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueErro